# Expamle use of AwsemLauncher

In [4]:
import os
import random 
import sys
sys.path.append('/work/shared/SimJobs')
from awsemjob import AwsemLauncher

Create a class as a child class from the parent AwsemLauncher with an abstract method <modify_parameters>

All the settings for the project and parameters for simulation jobs are defined in __init__

Use the function modify_parameters to control parameters for running each simulation.
Note even if there's nothing to be modified, the modify_parameters should always exist:

def modify_parameters(self, ijob, ifolder):
    pass
    
pbs file will be generated using the template defined in AwsemLauncher 

In [8]:
class MyProject(AwsemLauncher):
    
    current_dir = os.getcwd()
    
    def __init__(self, test=True):
        # project setting
        self.overwrite = True  # whether to overwrite project folder if it already exist
        self.verbose = True  # whether to print information
        self.work_dir = '/work/ace'  # the location of your project will be
        self.project_name = 'test'  # will create a project folder named as this in the work_dir
        self.run_id = 1  # to control restart run
        self.unfold = 100  # only used when run_id > 1
        self.njobs = 3  # max 99, number of independent simulations
        
        self.exe_path = '/work/shared/awsemmd/lmp_serial_g++_12182019'  # the location of your executable
        self.input_file = 'd1_Modeller_6n3c_12.in'  # the input file for the executable
        ## restart run will use this input file and modified it for a continued simulation
        
        self.test_submit = test  # to control whether to actually submit jobs
        
        # parameter files setting in work dir 
        ## copy files to work dir
        self.copy_files = ['test.txt', 'unBias_tmpFolder']  # could be folder or file
        
        # submit file setting
        self.submit_nodes = [1, 3]  # [1, 3]
        self.submit_ppn = [40, 20] # [10, 40]
        self.submit_cmd = self.exe_path + ' < ' + self.input_file
        ## cd to the work dir will be added automatically
    
    def modify_parameters(self, ijob, ifolder):
        # To replace str_a with str_b in file, use
        # self.modify_file(path, str_a, str_b, filename=None)
        self.modify_file(ifolder, 'banana', 'apple'+str(ijob), filename='test.txt')
        r = random.randint(1, 1e8) 
        self.modify_file(ifolder, 'velocity all create 300.0 2349852',
                         'velocity all create 300.0 %s' % r, filename=self.input_file)


In [11]:
# When test is set to True, all operations will be done except that no jobs will be actually submitted.
# Instead, it will print the submit command
myproject = MyProject(test=True)

In [12]:
myproject.run()

test submit: 
cd /work/ace/test/test_00/run_01 && qsub /work/ace/test/test_00/run_01/test.pbs
test submit: 
cd /work/ace/test/test_01/run_01 && qsub /work/ace/test/test_01/run_01/test.pbs
test submit: 
cd /work/ace/test/test_02/run_01 && qsub /work/ace/test/test_02/run_01/test.pbs
